# 阶段总结

## 简介

在第一阶段的学习和测试中，主要使用了openface，dlib，opencv等库中训练好的人脸识别模型，并在图片和实时视频上进行测试。同时，通过视频课程学习了基本CNN的结构（如padding，stride，pooling），ResNets对于深层CNN阶梯下降逐渐消失等问题的优化，YOLO的结构和算法概念（如IoU，Anchor Box，non-max suppression），以及相比于利用了region proposal的R-CNN，Fast/Faster R-CNN在实时运算上的优势。

## 学习过程

1. 尝试使用多种模型确定人脸及额头位置，并标出5点/68点人脸特征
    * 主要收获：
        * 熟悉detector和predictor的调用、输入、输出。
        * 通过返回的人脸特征点估算出额头的位置并用OpenCV的polylines方程画出额头区域。
2. 测试了dlib库中训练好的HOG模型确定人脸位置并标出5点/68点人脸特征
    * 主要收获：
        * 通过分别测试5点和68点人脸特征的fps，意识到两个人脸特征模型在运行速度上没有较大差距。
        * 通过比较两个模型文件大小，5点人脸特征模型的存储占用仅为68模型的1/10，从而得出5点模型更适合在移动设备上使用的结论。
3. 测试了ultra light模型确定人脸位置并标出5点/68点人脸特征
    * 主要收获：
        * 通过与dlib的HOG模型对比，ultra light的准确率、可检测面部角度、检测距离、已经运行速度都明显优于HOG。
        * 通过每n秒检测一次的方法进一步提高fps至25帧每秒。
4. 测试了KCF人脸跟踪模型与ultra light模型的融合：每n秒检测一次人脸位置并对跟踪校正，其他时候进行人脸跟踪
    * 主要收获：
        * 提高了人脸在持续移动时，人脸及额头选框的准确性。
        * 通过人脸跟踪算法来弥补非检测帧人脸移动带来的误差，可以进一步提高n值从而提高fps。

## 阶段性代码

In [ ]:
import numpy as np
import cv2
import dlib
import time
import onnx
import onnxruntime as ort
from onnx_tf.backend import prepare
from helpers import predict, get_forehead_coord, reshape_forehead_coord

import KCF

onDetecting = True
onTracking = False

onnx_path = 'model/ultra_light_320.onnx'  # OPTION: model/ultra_light_640.onnx
predictor_path = 'model/shape_predictor_5_face_landmarks.dat'
onnx_model = onnx.load(onnx_path)
detector = prepare(onnx_model)
predictor = dlib.shape_predictor(predictor_path)
ort_session = ort.InferenceSession(onnx_path)
input_name = ort_session.get_inputs()[0].name

trackers = []

cap = cv2.VideoCapture(0)

# initiate loop and timer
loop = 0
start = time.time()

while True:
    loop += 1

    ret, frame = cap.read()
    h, w, _ = frame.shape
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # convert bgr to grey

    if onDetecting:
        # pre-process img acquired
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert bgr to rgb
        img = cv2.resize(img, (320, 240))  # OPTION: 640 * 480
        img_mean = np.array([127, 127, 127])
        img = (img - img_mean) / 128
        img = np.transpose(img, [2, 0, 1])
        img = np.expand_dims(img, axis=0)
        img = img.astype(np.float32)

        confidences, boxes = ort_session.run(None, {input_name: img})
        boxes, labels, probs = predict(w, h, confidences, boxes, 0.7)

        for i in range(boxes.shape[0]):
            face_box = boxes[i, :]
            bX, bY, bW, bH = face_box
            cv2.rectangle(frame, (bX, bY), (bW, bH), (0, 255, 0), 2)
            tracker = KCF.kcftracker(False, True, False, False)  # hog, fixed_window, multi-scale, lab
            tracker.init([bX, bY, bW - bX, bH - bY], frame)
            trackers.append(tracker)

            face_rect = dlib.rectangle(left=bX, top=bY, right=bW, bottom=bH)
            shape = predictor(gray, face_rect)  # get 5-point facial landmarks
            cv2.polylines(frame, get_forehead_coord(shape), True, (0, 255, 255), 2)  # draw forehead box

        # keep detecting until face found
        if boxes.shape[0] > 0:
            onDetecting = False
            onTracking = True

    elif onTracking:
        for tracker in trackers:
            face_bbox = tracker.update(frame)  # get tracked face bounding box
            f_x1, f_y1, f_w, f_h = face_bbox[0], face_bbox[1], face_bbox[2], face_bbox[3]
            cv2.rectangle(frame, (f_x1, f_y1), (f_x1 + f_w, f_y1 + f_h), (0, 255, 0), 2)

            face_rect = dlib.rectangle(left=f_x1, top=f_y1, right=f_x1+f_w, bottom=f_y1+f_h)
            shape = predictor(gray, face_rect)  # get 5-point facial landmarks
            cv2.polylines(frame, get_forehead_coord(shape), True, (0, 255, 255), 2)  # draw forehead box

        # run detector every 10 frames
        if loop % 10 == 0:
            trackers = []
            forehead_trackers = []
            onDetecting = True
            onTracking = False

    cv2.imshow('tracking', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # end timer
        end = time.time()
        print("Average fps: ", loop / (end - start))
        break

cv2.destroyAllWindows()
cap.stop()

Github: https://github.com/andyzihaozhang/Face-Recog-and-Track-Dev

## 未来计划

1. 寻找合适的模型并通过fine-tune将模型应用到人脸口罩识别
    * 进展：
        * 已找到人脸口罩数据集
        * 已找到模型训练的[教程](https://www.pyimagesearch.com/2020/05/04/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/)，教程中使用了keras，TensorFlow等库和MobileNetV2模型（适合移动设备实时视频检测的模型）。
2. 将注意力转移到CT肺部分割
    * 进展：
        * 已找到多个已经训练好并有benchmark比较的模型（方老师提供）